# Calculation of inter-station paths for the visualization

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import LineString
import plotly.express as px

### Load the geojson data for Amtrak Northeast Corridor Stations with geopandas
* The `geometry` feature gives the (longitude, latitude) coordinate for each station of interest along the route of the Northeast Corridor
* The `STNCODE` feature gives the Amtrak station abbreviation

In [2]:
geo_stations = gpd.read_file('./data/geojson/Amtrak_Project_Stations.geojson')

In [3]:
geo_stations

,STNCODE,OBJECTID,STNNAME,CITY2,STATE,STFIPS,urban,geometry
0,BOS,5,"Boston (South Station), Massachusetts",Boston,MA,25,YES,POINT (-71.05530 42.35231)
1,BBY,17,"Boston (Back Bay), Massachusetts",Boston,MA,25,YES,POINT (-71.07583 42.34732)
2,RTE,24,"Westwood, Route 128 Station, Massachusetts",Route 128,MA,25,YES,POINT (-71.14789 42.21024)
3,PVD,10,"Providence, Rhode Island",Providence,RI,44,YES,POINT (-71.41348 41.82949)
4,KIN,60,"West Kingston, Rhode Island",Kingston,RI,44,,POINT (-71.56060 41.48396)
5,NLC,63,"New London, Connecticut",New London,CT,09,YES,POINT (-72.09322 41.35427)
6,NHV,16,"New Haven, Connecticut",New Haven,CT,09,YES,POINT (-72.92667 41.29771)
7,STM,27,"Stamford, Connecticut",Stamford,CT,09,YES,POINT (-73.54216 41.04713)
8,NYP,1,"New York (Penn Station), New York",New York,NY,36,YES,POINT (-73.99446 40.75033)
9,NWK,14,"Newark (Penn Station), New Jersey",Newark,NJ,34,YES,POINT (-74.16475 40.73471)


### Define stations and mile markers for each station going in the Northbound and Southbound directions

In [4]:
amtrak_stations = ['BOS', 'BBY', 'RTE', 'PVD', 'KIN', 'NLC', 'NHV', 'STM',
                   'NYP', 'NWK', 'TRE', 'PHL', 'WIL', 'BAL', 'BWI', 'NCR', 'WAS']

mile_markers = {'Northbound Mile': {station: None for station in amtrak_stations} , 'Southbound Mile': {station: None for station in amtrak_stations} } 

# Mile markers for each station along route starting in Boston and heading to DC
SB = [0, 1, 11, 43, 70, 105, 156, 195, 231, 241, 289, 322, 347, 416, 427, 448, 457]

# Mile markers for each station along route starting in DC and heading to Boston
NB = [457, 456, 446, 414, 387, 352, 301, 262, 226, 216, 168, 135, 110, 41, 30, 9, 0]

# Add to dictionary and then create data frame
for station, NB_mile, SB_mile in zip(amtrak_stations, NB, SB):
    mile_markers['Northbound Mile'][station] = NB_mile
    mile_markers['Southbound Mile'][station] = SB_mile

mile_cols = pd.DataFrame.from_dict(mile_markers, orient='columns')

### Create data frame of longitude/latitude values, indexed by Amtrak station code

In [5]:
lons = pd.Series(geo_stations.geometry.x, name='LON')
lats =  pd.Series(geo_stations.geometry.y, name='LAT')
gdf_lonlat = pd.DataFrame([lons, lats]).T
gdf_lonlat.set_index(geo_stations['STNCODE'], inplace=True)
gdf_lonlat

,LON,LAT
STNCODE,,
BOS,-71.055304,42.352311
BBY,-71.075828,42.347317
RTE,-71.147894,42.210242
PVD,-71.413478,41.829490
KIN,-71.560597,41.483959
NLC,-72.093225,41.354267
NHV,-72.926670,41.297714
STM,-73.542160,41.047130
NYP,-73.994459,40.750327


### Create new data frame indexed by Amtrak station codes
* Contains columns for longitude, latitude, mile markers, other relevant info

In [6]:
# Extract desired columns
gdf = geo_stations[['STNCODE', 'STNNAME', 'CITY2', 'STATE']]
# Reindex using Amtrak station names
gdf = gdf.set_index(gdf['STNCODE']).drop('STNCODE', axis=1)
# Combine with longitude and latitude and mile markers data
gdf = pd.concat([gdf, gdf_lonlat, mile_cols], axis = 1)
# Rearrange based on order along route
gdf = gdf.loc[amtrak_stations]

In [7]:
gdf

,STNNAME,CITY2,STATE,LON,LAT,Northbound Mile,Southbound Mile
BOS,"Boston (South Station), Massachusetts",Boston,MA,-71.055304,42.352311,457,0
BBY,"Boston (Back Bay), Massachusetts",Boston,MA,-71.075828,42.347317,456,1
RTE,"Westwood, Route 128 Station, Massachusetts",Route 128,MA,-71.147894,42.210242,446,11
PVD,"Providence, Rhode Island",Providence,RI,-71.413478,41.829490,414,43
KIN,"West Kingston, Rhode Island",Kingston,RI,-71.560597,41.483959,387,70
NLC,"New London, Connecticut",New London,CT,-72.093225,41.354267,352,105
NHV,"New Haven, Connecticut",New Haven,CT,-72.926670,41.297714,301,156
STM,"Stamford, Connecticut",Stamford,CT,-73.542160,41.047130,262,195
NYP,"New York (Penn Station), New York",New York,NY,-73.994459,40.750327,226,231
NWK,"Newark (Penn Station), New Jersey",Newark,NJ,-74.164750,40.734706,216,241


### Process to determine the station paths
* Store info in a dictionary

In [8]:
# Extract latitude and longitude columns and stations index and create dictionary for storage
lat = gdf['LAT']
lon = gdf['LON']
stations = gdf.index
geoloc_dict = {station: {'x': None, 'y': None, 'prev': None, 
                         'next': None, 'path2next': None, 'path2prev': None, 'index_where': None} for station in stations}

# For each station, set the x and y coordinates to the longitude and latitude values
# Also set the previous and next stations along the route (using Southbound-orient indexing)
prev_stat = None
for i, station in enumerate(stations):
    geoloc_dict[station]['x'] = lon[station] 
    geoloc_dict[station]['y'] = lat[station] 
    geoloc_dict[station]['prev'] = prev_stat
    if station != 'WAS':
        geoloc_dict[station]['next'] = stations[i+1]
    prev_stat = station
    print(station, 'lon:', geoloc_dict[station]['x'], 'lat:', geoloc_dict[station]['y'], 
          'prev:', geoloc_dict[station]['prev'], 'next:', geoloc_dict[station]['next'])

BOS lon: -71.05530399962942 lat: 42.35231100012763 prev: None next: BBY
BBY lon: -71.07582800026684 lat: 42.34731700021211 prev: BOS next: RTE
RTE lon: -71.14789400005567 lat: 42.21024199995967 prev: BBY next: PVD
PVD lon: -71.4134779996708 lat: 41.829490000162025 prev: RTE next: KIN
KIN lon: -71.5605969999761 lat: 41.483959000033835 prev: PVD next: NLC
NLC lon: -72.09322499959666 lat: 41.35426700011465 prev: KIN next: NHV
NHV lon: -72.92666999955892 lat: 41.29771399982176 prev: NLC next: STM
STM lon: -73.54215999966355 lat: 41.047130000267344 prev: NHV next: NYP
NYP lon: -73.99445899996624 lat: 40.75032699989306 prev: STM next: NWK
NWK lon: -74.16474999990626 lat: 40.73470599995154 prev: NYP next: TRE
TRE lon: -74.75443999967372 lat: 40.21901100002319 prev: NWK next: PHL
PHL lon: -75.18104099974647 lat: 39.95561500001248 prev: TRE next: WIL
WIL lon: -75.55109500023144 lat: 39.737262999896686 prev: PHL next: BAL
BAL lon: -76.61568799967372 lat: 39.30730200018028 prev: WIL next: BWI
BWI

### Creating a list of coordinates for latitude and longitude values
* Will convert to CSV format rather than GeoJSON

In [9]:
geo_route = gpd.read_file('./data/geojson/Amtrak_Project_Routes.geojson')
geo_route

,OBJECTID,NAME,Shape_Leng,Shape_Le_1,Shape_Length,geometry
0,29,Regional,1.041187e+06,1.041187e+06,1.358052e+06,"MULTILINESTRING ((-76.45187 37.02302, -76.4527..."


### Add all latitude/longitude coordinates to individual lists

In [10]:
lats = []
lons = []
for mlstring in geo_route.geometry:
    linestrings = mlstring.geoms
    for linestring in linestrings: 
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
print(len(lats), len(lons))

13258 13258


### Reverse them to Southbound orientation

In [11]:
lons = lons[::-1]
lats = lats[::-1]

### Extract only Boston to Washington (exclude Virginia coordinates)

In [12]:
lats_bos_to_was = []
lons_bos_to_was = []
for x, y in zip(lons, lats):
    if x < -77.006422 and y < 38.896993:
        break
    else:
        lats_bos_to_was = np.append(lats_bos_to_was, y)
        lons_bos_to_was = np.append(lons_bos_to_was, x)
print(len(lons_bos_to_was))
print(len(lats_bos_to_was))

9550
9550


### Convert to numpy array and then data frame for later use

In [13]:
lonlat_bos_to_was = np.array([lons_bos_to_was, lats_bos_to_was]).T
print(lonlat_bos_to_was.shape)
lonlat_df = pd.DataFrame(lonlat_bos_to_was, columns = ["Longitude","Latitude"])
lonlat_df.tail()

(9550, 2)


,Longitude,Latitude
9545,-77.005867,38.886143
9546,-77.006019,38.885972
9547,-77.006110,38.885900
9548,-77.006242,38.885795
9549,-77.006305,38.885746


### Calculate the indices where the longitude and latitude are within a small tolerance distance of the station
* Coordinates from the original file do not include Boston-South Station, they only have data to/from Boston-Back Bay
* The tolerance is relatively large but further precision would exclude some stations because they have 0 points within the smaller range. 

In [14]:
for station in amtrak_stations:
    station_x, station_y = geoloc_dict[station]['x'], geoloc_dict[station]['y']
    prev_stat, next_stat = geoloc_dict[station]['prev'], geoloc_dict[station]['next']
    where_near_x = np.argwhere(np.isclose(lons, station_x, atol = 0.001))
    where_near_y = np.argwhere(np.isclose(lats, station_y, atol = 0.001))
    intersect = np.intersect1d(where_near_x, where_near_y)
    geoloc_dict[station]['index_where'] = intersect
    print(intersect)

[]
[0 1 2 3 4 5 6 7]
[152 153 154 155 156 157 158 159 160 161]
[652 653 654 655 656 657 658 659]
[1025 1026 1027 1028]
[1810 1811 1812 1813 1814 1815 1816 1817 1818 1819]
[3185]
[4058 4059]
[5067 5068 5069 5070 5071]
[5289 5290 5291 5292 5313 5314 5315 5316]
[5947 5948]
[6664 6665 6666 6667 6668 6669 6670 6671 6672 6673 6674 6675 6676 6677
 6678 6679 6680]
[7283 7284 7285 7286 7287 7288 7289 7290 7291 7292]
[8579 8580 8581]
[8900 8901 8902 8903]
[9242 9243 9244]
[9510 9511]


In [15]:
print(lonlat_df.iloc[9509]) # Verify last point

Longitude   -77.00414
Latitude     38.89975
Name: 9509, dtype: float64


In [16]:
lonlat_df = lonlat_df.iloc[0:9510] # Select only relevant coordinates

### Calculate inter-station paths to find "path groups" for the visualization separators

In [17]:
# For each station, add the group indicator to a list for a new column in the dataframe
between_station_paths = []
group_num = 0
groups_path = []
for curr_stat in amtrak_stations:
    if curr_stat == 'BOS':
        continue
    prev_stat, next_stat = geoloc_dict[curr_stat]['prev'], geoloc_dict[curr_stat]['next']
    if next_stat is not None:
        curr_stat_index_where = geoloc_dict[curr_stat]['index_where'][0]
        next_stat_index_where = geoloc_dict[next_stat]['index_where'][0]
        path2next = []
        for i in range(curr_stat_index_where, next_stat_index_where):
            between_station_paths.append('{}-{}'.format(curr_stat, next_stat))
            path2next.append(group_num)
            groups_path.append(group_num)
        geoloc_dict[curr_stat]['path2next'] = path2next
        group_num += 1
    if prev_stat is not None:
        geoloc_dict[curr_stat]['path2prev'] = geoloc_dict[prev_stat]['path2next']
print(len(groups_path))

9510


In [18]:
print(len(groups_path))
print(lonlat_df.shape[0])

9510
9510


### Create series to add to dataframe

In [19]:
group_num = pd.Series(groups_path, name = 'Group')
group_text = pd.Series(between_station_paths, name = 'Connecting Path')
print(group_text.shape[0])
print(group_num.shape[0])

9510
9510


In [20]:
lonlat_with_path_groups = pd.concat([lonlat_df, group_num, group_text], axis = 1)

In [21]:
print(lonlat_with_path_groups[145:170])

     Longitude   Latitude  Group Connecting Path
145 -71.138449  42.228231      0         BBY-RTE
146 -71.138671  42.227777      0         BBY-RTE
147 -71.139009  42.227083      0         BBY-RTE
148 -71.139243  42.226603      0         BBY-RTE
149 -71.140700  42.223612      0         BBY-RTE
150 -71.143650  42.217497      0         BBY-RTE
151 -71.145931  42.212768      0         BBY-RTE
152 -71.146592  42.211393      1         RTE-PVD
153 -71.146702  42.211163      1         RTE-PVD
154 -71.146747  42.211071      1         RTE-PVD
155 -71.146996  42.210552      1         RTE-PVD
156 -71.147195  42.210138      1         RTE-PVD
157 -71.147200  42.210128      1         RTE-PVD
158 -71.147221  42.210082      1         RTE-PVD
159 -71.147346  42.209823      1         RTE-PVD
160 -71.147380  42.209752      1         RTE-PVD
161 -71.147599  42.209295      1         RTE-PVD
162 -71.147935  42.208601      1         RTE-PVD
163 -71.148058  42.208281      1         RTE-PVD
164 -71.148171  42.2

In [22]:
lonlat_with_path_groups

,Longitude,Latitude,Group,Connecting Path
0,-71.075149,42.347551,0,BBY-RTE
1,-71.075410,42.347485,0,BBY-RTE
2,-71.075579,42.347441,0,BBY-RTE
3,-71.075729,42.347391,0,BBY-RTE
4,-71.075788,42.347366,0,BBY-RTE
...,...,...,...,...
9505,-77.003981,38.904034,14,NCR-WAS
9506,-77.004064,38.903772,14,NCR-WAS
9507,-77.004043,38.901542,14,NCR-WAS
9508,-77.004053,38.900194,14,NCR-WAS


### Export everything to CSV

In [23]:
lonlat_with_path_groups.to_csv("./data/visualization/NE_regional_lonlat.csv", index = False)

In [24]:
gdf['STNCODE'] = gdf.index
gdf = gdf.reset_index().drop('index', axis = 1)

In [25]:
gdf.to_csv('./data/visualization/geo_stations_info.csv', index = False)